# Converting [Whp](http://www.roy.hi-ho.ne.jp/ssai/mito_gis/whpview/index.html) (JMA compatible) format to [GWO](http://www.roy.hi-ho.ne.jp/ssai/mito_gis/) format
**Author: Jun Sasaki  Coded on 2022-02-13  Updated on 2024-07-07**<br>
気象庁互換形式でウェザートーイのサポートから配布されているCSVデータを気象庁地上観測データGWO形式のCSVデータに変換する．両者で単位や表記が異なるため，GWO形式に合わせる．時別値に対応済みである．日別値とAMEDASにも対応したい．

- Note: pvlib required:
```bash
conda install pvlib
```

## 気象庁互換形式
- 2022年以降の地上気象観測データとアメダスデータは[ウェザートーイ](http://www.roy.hi-ho.ne.jp/ssai/mito_gis/)のサポートにおいて，気象庁互換形式で配布されている．気象庁互換形式をGWO形式に変換する．
- 気象庁互換形式の値欄には記号が含まれている場合がある．値欄の記号の詳細は[こちら](https://www.data.jma.go.jp/obd/stats/data/mdrr/man/remark.html)．

### 値欄の処理
- 雲量に`0+`がある．また，雲量は3時間データであるため，データの無いところはNaNにしておく必要がある．
- 降水量の `--` とデータ無しは欠損値とし，欠損値はすべて0とする．Remarkが存在しないため，欠損値を0とする以外の方法はない．

```
降雪，積雪：削除
雲量： '0+' => 0
降水量： '--' => 0
日照時間： '' => 0
全天日射： '' => 0 （全天日射量が観測されていない測点では0ではなくNaNにする）
降雪(cm)，積雪(cm)，--，視程：削除
["現地気圧(hPa)","海面気圧(hPa)","降水量(mm)","気温(℃)","露点温度(℃)","蒸気圧(hPa)","風速(m/s)","日照時間(h)","全天日射量(MJ/㎡)"]
```

In [ ]:
from metdata.whp import Whp, StationDict

## Create Whp instance for whp CSV (JMA compatible) in GWO compatible CSV
- The created instance is a dictionary of `whp[stn, year]`
- pandas.DataFrame is obtained by `whp[stn, year].df` 
- Specify `data_dir_base` where CSV_file path is `data_dir_base/stn/year/CSV_file`. 
- Specify stations list `stns` and years list `years`.
### Specify input and output directory bases, stations `stns`, and years `years`

In [ ]:
whp_dir_base = '/mnt/d/dat/met/JMA_DataBase/whpView/wwwRoot/data/'
gwo_dir_base = "/mnt/d/dat/met/JMA_DataBase/GWO/Hourly/"
stns = ['Tokyo','Yokohama', 'Chiba']
years = [2022]
whp_data = StationDict()

### Create Whp instances

In [ ]:
for stn in stns: 
    for year in years:
        whp_data[(stn, year)]=Whp(year=year, stn=stn, data_dir_base=whp_dir_base)

In [ ]:
'''
#whp_data['Yokohama', 2022].df.iloc[3901,:]
whp_data['Yokohama', 2022].df.head()
'''

### Export Whp CSV files in GWO format

In [ ]:
for stn in stns:
    for year in years:
        whp_data[stn, year].to_csv(data_dir_base=gwo_dir_base, isfpath=False)
